In [1]:
from ngsolve import *
from ngsolve.meshes import Make1DMesh
from ngsolve.la import EigenValues_Preconditioner
from ngsolve.webgui import Draw
import numpy as np
import math
import scipy.linalg
from scipy import random

In [2]:
mesh = Make1DMesh(40)
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [3]:
w = H1(mesh, order=2, dirichlet="left|right")
sigma = H1(mesh, order=2, dirichlet=" ")
fesm = w*sigma

In [4]:
w, sigma = fesm.TrialFunction()
v , tau = fesm.TestFunction()

In [5]:
a = BilinearForm(fesm,symmetric=True)
a += (-grad(sigma)*grad(v) + sigma*tau + grad(w)*grad(tau))*dx
pre = Preconditioner(a,"bddc")
# pre = Preconditioner(a,"multigrid")

m = BilinearForm(fesm)
m += grad(w)*grad(v)*dx

a.Assemble()
m.Assemble()

u = GridFunction(fesm)

In [6]:
r = u.vec.CreateVector()
w = u.vec.CreateVector()
Mu = u.vec.CreateVector()
Au = u.vec.CreateVector()
Mw = u.vec.CreateVector()
Aw = u.vec.CreateVector()

In [7]:
# r.FV().NumPy()[:] = random.rand(fesm.ndof)
r.FV().NumPy()[:] = np.ones(fesm.ndof)
u.vec.data = Projector(fesm.FreeDofs(), True) * r

In [10]:
for i in range(1):
    Au.data = a.mat * u.vec
    Mu.data = m.mat * u.vec
    auu = InnerProduct(Au, u.vec)
    muu = InnerProduct(Mu, u.vec)
    # Rayleigh quotient
    lam = auu/muu
#     print (lam / (math.pi**2))
    # residual
    r.data = Au - lam * Mu
    w.data = pre.mat * r.data
    w.data = 1/Norm(w) * w
    Aw.data = a.mat * w
    Mw.data = m.mat * w

    # setup and solve 2x2 small eigenvalue problem
    asmall = Matrix(2,2)
    asmall[0,0] = auu
    asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
    asmall[1,1] = InnerProduct(Aw, w)
    msmall = Matrix(2,2)
    msmall[0,0] = muu
    msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
    msmall[1,1] = InnerProduct(Mw, w)
    # print ("asmall =", asmall, ", msmall = ", msmall)
    
    
    eval_,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    print (eval_)
    print(eval_[0])
    print(eval_[1])
    print(evec)
    print(evec[0,:])
    print(evec[1,:])

[0.00342534 0.01182291]
0.003425336093603409
0.011822912423622569
[[-0.07950834 -0.03432818]
 [ 0.30161455  1.17397115]]
[-0.07950834 -0.03432818]
[0.30161455 1.17397115]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import array
from numpy import linalg as LA

# Assume that I loaded 'N' no of 2d points from a file and used
# np.cov() to find the below covariance matrix

# This is my covariance matrix obtained from 2 x N points
cov_mat = [[3407.3108669,  1473.06388943],
           [1473.06388943, 1169.53151003]]

eigen_values, eigen_vectors = LA.eig(cov_mat)

origin = [0, 0]

eig_vec1 = eigen_vectors[:,0]
eig_vec2 = eigen_vectors[:,1]

print(eig_vec1)
print(eig_vec2)


# This line below plots the 2d points
#plt.scatter(np_array[:,0], np_array[:,1])

plt.quiver(*origin, *eig_vec1, color=['r'], scale=21)
plt.quiver(*origin, *eig_vec2, color=['b'], scale=21)
plt.show()